## Search algorithms within Optuna

In this notebook, I will demo how to select the search algorithm with Optuna. We will compare the use of:

- Grid Search 
- Randomized search
- Tree-structured Parzen Estimators
- CMA-ES


We can select the search algorithm from the [optuna.study.create_study()](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.study.create_study.html#optuna.study.create_study) class.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

import optuna

In [3]:
# load dataset

breast_cancer_X, breast_cancer_y = load_breast_cancer(return_X_y=True)
X = pd.DataFrame(breast_cancer_X)
y = pd.Series(breast_cancer_y).map({0:1, 1:0})

X.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
# the target:
# percentage of benign (0) and malign tumors (1)

y.value_counts() / len(y)

0    0.627417
1    0.372583
dtype: float64

In [5]:
# split dataset into a train and test set

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((398, 30), (171, 30))

## Define the objective function

This is the hyperparameter response space, the function we want to minimize.

In [6]:
# the objective function takes the hyperparameter space
# as input

def objective(trial):

    rf_n_estimators = trial.suggest_int("rf_n_estimators", 100, 1000)
    rf_criterion = trial.suggest_categorical("rf_criterion", ['gini', 'entropy'])
    rf_max_depth = trial.suggest_int("rf_max_depth", 1, 4)
    rf_min_samples_split = trial.suggest_float("rf_min_samples_split", 0.01, 1)
    
    model = RandomForestClassifier(
        n_estimators=rf_n_estimators,
        criterion=rf_criterion,
        max_depth=rf_max_depth,
        min_samples_split=rf_min_samples_split,
    )

    score = cross_val_score(model, X_train, y_train, cv=3)
    accuracy = score.mean()
    return accuracy

## Randomized Search

RandomSampler()

In [8]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.RandomSampler(),
)


study.optimize(objective, n_trials=10)

[I 2022-10-07 16:24:23,355] A new study created in memory with name: no-name-5856b91f-7972-40cc-a961-b9d60077da19
[I 2022-10-07 16:24:24,399] Trial 0 finished with value: 0.9196476038581302 and parameters: {'rf_n_estimators': 600, 'rf_criterion': 'gini', 'rf_max_depth': 1, 'rf_min_samples_split': 0.600115890802639}. Best is trial 0 with value: 0.9196476038581302.
[I 2022-10-07 16:24:25,441] Trial 1 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 707, 'rf_criterion': 'entropy', 'rf_max_depth': 2, 'rf_min_samples_split': 0.6762068975852243}. Best is trial 0 with value: 0.9196476038581302.
[I 2022-10-07 16:24:26,986] Trial 2 finished with value: 0.9296157059314955 and parameters: {'rf_n_estimators': 805, 'rf_criterion': 'gini', 'rf_max_depth': 4, 'rf_min_samples_split': 0.3570405061717201}. Best is trial 2 with value: 0.9296157059314955.
[I 2022-10-07 16:24:28,178] Trial 3 finished with value: 0.9146350725298094 and parameters: {'rf_n_estimators': 709, 'rf_crit

In [9]:
study.best_params

{'rf_n_estimators': 627,
 'rf_criterion': 'gini',
 'rf_max_depth': 2,
 'rf_min_samples_split': 0.020709626160625143}

In [10]:
study.best_value

0.9397357028935976

In [11]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_rf_criterion,params_rf_max_depth,params_rf_min_samples_split,params_rf_n_estimators,state
0,0,0.919648,2022-10-07 16:24:23.358145,2022-10-07 16:24:24.399010,0 days 00:00:01.040865,gini,1,0.600116,600,COMPLETE
1,1,0.625636,2022-10-07 16:24:24.400075,2022-10-07 16:24:25.441576,0 days 00:00:01.041501,entropy,2,0.676207,707,COMPLETE
2,2,0.929616,2022-10-07 16:24:25.442572,2022-10-07 16:24:26.986688,0 days 00:00:01.544116,gini,4,0.357041,805,COMPLETE
3,3,0.914635,2022-10-07 16:24:26.987651,2022-10-07 16:24:28.178564,0 days 00:00:01.190913,gini,4,0.610861,709,COMPLETE
4,4,0.934666,2022-10-07 16:24:28.179643,2022-10-07 16:24:28.600533,0 days 00:00:00.420890,gini,2,0.312258,227,COMPLETE
5,5,0.939736,2022-10-07 16:24:28.601462,2022-10-07 16:24:29.762421,0 days 00:00:01.160959,gini,2,0.020710,627,COMPLETE
6,6,0.625636,2022-10-07 16:24:29.763407,2022-10-07 16:24:31.128043,0 days 00:00:01.364636,entropy,4,0.791016,921,COMPLETE
7,7,0.919648,2022-10-07 16:24:31.129048,2022-10-07 16:24:32.463516,0 days 00:00:01.334468,entropy,3,0.546149,774,COMPLETE
8,8,0.625636,2022-10-07 16:24:32.464642,2022-10-07 16:24:32.878965,0 days 00:00:00.414323,gini,3,0.832165,272,COMPLETE
9,9,0.625636,2022-10-07 16:24:32.880106,2022-10-07 16:24:33.468027,0 days 00:00:00.587921,gini,1,0.807726,387,COMPLETE


## TPE

TPESampler is the default

In [12]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
)


study.optimize(objective, n_trials=5)

[I 2022-10-07 16:24:52,260] A new study created in memory with name: no-name-f1298ac9-77d3-4502-b16c-43798b7dcff8
[I 2022-10-07 16:24:53,617] Trial 0 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 899, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_samples_split': 0.7077732992599175}. Best is trial 0 with value: 0.6256360598465861.
[I 2022-10-07 16:24:54,673] Trial 1 finished with value: 0.9221728563833826 and parameters: {'rf_n_estimators': 632, 'rf_criterion': 'entropy', 'rf_max_depth': 1, 'rf_min_samples_split': 0.30024577491324306}. Best is trial 1 with value: 0.9221728563833826.
[I 2022-10-07 16:24:55,164] Trial 2 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 333, 'rf_criterion': 'gini', 'rf_max_depth': 2, 'rf_min_samples_split': 0.9543522925471329}. Best is trial 1 with value: 0.9221728563833826.
[I 2022-10-07 16:24:56,345] Trial 3 finished with value: 0.9321409584567478 and parameters: {'rf_n_estimators': 620, 'rf_cr

In [13]:
study.best_params

{'rf_n_estimators': 620,
 'rf_criterion': 'gini',
 'rf_max_depth': 4,
 'rf_min_samples_split': 0.3292567147048957}

In [14]:
study.best_value

0.9321409584567478

## CMA-ES

CmaEsSampler

In [15]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.CmaEsSampler(),
)

study.optimize(objective, n_trials=5)

[I 2022-10-07 16:25:41,333] A new study created in memory with name: no-name-ef5ccf58-697a-4a1a-8c4e-afe845862f35
[I 2022-10-07 16:25:43,180] Trial 0 finished with value: 0.9472544998860788 and parameters: {'rf_n_estimators': 862, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 0.019005506756964653}. Best is trial 0 with value: 0.9472544998860788.
[W 2022-10-07 16:25:43,189] The parameter 'rf_criterion' in trial#1 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[I 2022-10-07 16:25:44,010] Trial 1 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 550, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 0

In [16]:
study.best_params

{'rf_n_estimators': 862,
 'rf_criterion': 'entropy',
 'rf_max_depth': 3,
 'rf_min_samples_split': 0.019005506756964653}

In [17]:
study.best_value

0.9472544998860788

## Grid Search

GridSampler()

We are probably not going to perform GridSearch with Optuna, but in case you wanted to, you need to add a variable with the space, with the exact values that you want to be tested.

In [18]:
search_space = {
    "rf_n_estimators": [100, 500, 1000],
    "rf_criterion": ['gini', 'entropy'],
    "rf_max_depth": [1, 2, 3],
    "rf_min_samples_split": [0.1, 1.0]
}


In [19]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.GridSampler(search_space),
)

study.optimize(objective)

[I 2022-10-07 16:26:15,882] A new study created in memory with name: no-name-eab4ef2e-fc7a-4000-ae04-947cb225b789
[I 2022-10-07 16:26:16,659] Trial 0 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 500, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 1.0}. Best is trial 0 with value: 0.6256360598465861.
[I 2022-10-07 16:26:18,509] Trial 1 finished with value: 0.9397357028935976 and parameters: {'rf_n_estimators': 1000, 'rf_criterion': 'gini', 'rf_max_depth': 2, 'rf_min_samples_split': 0.1}. Best is trial 1 with value: 0.9397357028935976.
[I 2022-10-07 16:26:19,507] Trial 2 finished with value: 0.937210450368345 and parameters: {'rf_n_estimators': 500, 'rf_criterion': 'entropy', 'rf_max_depth': 2, 'rf_min_samples_split': 0.1}. Best is trial 1 with value: 0.9397357028935976.
[I 2022-10-07 16:26:21,017] Trial 3 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 1000, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_m

In [20]:
study.best_params

{'rf_n_estimators': 100,
 'rf_criterion': 'entropy',
 'rf_max_depth': 2,
 'rf_min_samples_split': 0.1}

In [21]:
study.best_value

0.942241968557758